<a href="https://colab.research.google.com/github/shamaryr4/parqueadero_nido_de_ruedas/blob/main/final_segunda_entrega_programaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
#importar
import re
from datetime import datetime, timedelta

#precios
ESPACIOS_TOTALES = 64
PRECIO_HORA = 7000
PRECIO_CUARTO = 1500

#listas
usuarios = []
vehiculos_registrados = {}
vehiculos_retirados = []
administradores = {"admin": "1234"}

#nombre del cliente validando restricciones
def validar_nombre(nombre):
    errores = []
    if len(nombre) < 3:
        errores.append("El nombre debe tener al menos 3 letras.")

    contiene_numero = False
    for char in nombre:
        try:
            int(char)
            contiene_numero = True
            break
        except ValueError:
            continue

    if contiene_numero:
        errores.append("El nombre no debe contener números.")

    return errores

#apellido del cliente validando restricciones
def validar_apellido(apellido):
    errores = []

    if len(apellido) < 3:
        errores.append("El apellido debe tener al menos 3 letras.")

    contiene_numero = False
    for char in apellido:
        try:
            int(char)  # Intentamos convertir el carácter a número
            contiene_numero = True
            break      # Si lo logramos, hay un número, salimos del ciclo
        except ValueError:
            continue   # Si no es un número, seguimos con el siguiente carácter

    if contiene_numero:
        errores.append("El apellido no debe contener números.")

    return errores

#cedula del cliente validando restricciones
def validar_documento(documento):
    errores = []
    if not documento.isdigit():
        errores.append("El documento solo debe contener números.")
    if not 3 <= len(documento) <= 15:
        errores.append("El documento debe tener entre 3 y 15 dígitos.")
    return errores

#placa del vehiculo validando restricciones
def validar_placa(placa):
    errores = []
    if len(placa) != 6:
        errores.append("La placa debe tener exactamente 6 caracteres.")
    elif not (placa[:3].isalpha() and placa[3:].isdigit()):
        errores.append("La placa debe tener 3 letras seguidas de 3 números.")
    return errores

#registro del vehiculo
def registrar_vehiculo():
    if len(vehiculos_registrados) >= ESPACIOS_TOTALES:
        print("Parqueadero lleno. No hay espacios disponibles.")
        return

    nombre = input("Ingrese nombre: ")
    apellido = input("Ingrese apellido: ")
    documento = input("Ingrese documento: ")
    placa = input("Ingrese placa del vehículo: ").upper()

    errores = validar_nombre(nombre) + validar_apellido(apellido) + \
              validar_documento(documento) + validar_placa(placa)

    if errores:
        print("\nErrores en el ingreso de datos:")
        for error in errores:
            print("- {}".format(error))
        return

    hora_entrada = datetime.now()
    vehiculos_registrados[placa] = {
        "nombre": nombre,
        "apellido": apellido,
        "documento": documento,
        "hora_entrada": hora_entrada
    }
    usuarios.append({"nombre": nombre, "apellido": apellido, "documento": documento})

    print("\nVehículo ingresado exitosamente.\nRecibo:\nPlaca: {}\nHora de entrada: {}\n".format(placa, hora_entrada.strftime('%H:%M') ))

#retiro del vehiculo
def retirar_vehiculo():
    placa = input("Ingrese la placa del vehículo a retirar: ").upper()
    if placa not in vehiculos_registrados:
        print("\nEl vehículo no está registrado.\n")
        return

    entrada = vehiculos_registrados.pop(placa)
    salida = datetime.now()
    tiempo_total = salida - entrada['hora_entrada']

    horas = tiempo_total.seconds // 3600
    cuartos = ((tiempo_total.seconds % 3600) + 899) // 900  # redondeo hacia arriba
    total = horas * PRECIO_HORA + cuartos * PRECIO_CUARTO
    total = max(total, PRECIO_HORA)

    vehiculos_retirados.append({"placa": placa, "entrada": entrada['hora_entrada'], "salida": salida, "total": total})

    print("\nFactura de retiro:\nPlaca: {}\nHora entrada: {}\nHora salida: {}\nTotal a pagar: ${}\n".format(
    placa,
    entrada['hora_entrada'].strftime('%H:%M'),
    salida.strftime('%H:%M'),
    total))

#ingresar menu de administrador
def menu_administrador():
    usuario = input("Ingrese usuario administrador: ")
    clave = input("Ingrese clave: ")

    if administradores.get(usuario) != clave:
        print("\nUsuario o clave incorrectos.\n")
        return

    while True:
        print("""
--- Reportes Administrador ---
1. Total de vehículos registrados
2. Total de vehículos retirados
3. Total de vehículos sin retirar
4. Total pagado por vehículos retirados
5. Tiempo promedio de parqueo
6. Lista de usuarios
7. Vehículo con tiempo máximo y mínimo
8. Salir
        """)
        opcion = input("Seleccione una opción: ")

        if opcion == "1":
           print("Total de vehículos registrados: {}".format(len(vehiculos_registrados)))
        elif opcion == "2":
           print(f"Total de vehículos retirados: {len(vehiculos_retirados)}")
        elif opcion == "3":
            print(f"Total de vehículos sin retirar: ${len(vehiculos_registrados)}")
        elif opcion == "4":
            total = sum(v["total"] for v in vehiculos_retirados)
            print("Total pagado por vehículos retirados: ${}".format(total))
        elif opcion == "5":
            if vehiculos_retirados:
                promedio = sum((v["salida"] - v["entrada"]).seconds for v in vehiculos_retirados) / len(vehiculos_retirados)
                print("Tiempo promedio de parqueo: {} minutos".format(int(promedio // 60)))
            else:
                print("No hay datos de vehículos retirados.")
        elif opcion == "6":
            for u in usuarios:
                print("{} {} - Documento: {}".format(u['nombre'], u['apellido'], u['documento']))
        elif opcion == "7":
            if not vehiculos_retirados:
                print("No hay vehículos retirados.")
                continue
            tiempos = [(v["placa"], (v["salida"] - v["entrada"]).seconds) for v in vehiculos_retirados]
            max_veh = max(tiempos, key=lambda x: x[1])
            min_veh = min(tiempos, key=lambda x: x[1])
            print("Vehículo con mayor tiempo: {} - {} minutos".format(max_veh[0],max_veh[1] // 60))
            print("Vehículo con menor tiempo: {} - {} minutos".format(min_veh[0], min_veh[1] // 60))
        elif opcion == "8":
            break
        else:
            print("Opción inválida.")

#menu principal
def menu_principal():
    while True:
        print("""
--- Parqueadero Nido de Ruedas ---
1. Ingresar vehículo
2. Retirar vehículo
3. Módulo administrador
4. Salir
        """)
        opcion = input("Seleccione una opción: ")

        if opcion == "1":
            registrar_vehiculo()
        elif opcion == "2":
            retirar_vehiculo()
        elif opcion == "3":
            menu_administrador()
        elif opcion == "4":
            print("Gracias por usar el sistema del parqueadero.")
            break
        else:
            print("Opción inválida.")

#ejecutar
if __name__ == "__main__":
    menu_principal()



--- Parqueadero Nido de Ruedas ---
1. Ingresar vehículo
2. Retirar vehículo
3. Módulo administrador
4. Salir
        
Seleccione una opción: 1
Ingrese nombre: sdafsd
Ingrese apellido: fsda
Ingrese documento: 564s
Ingrese placa del vehículo: safd

Errores en el ingreso de datos:
- El documento solo debe contener números.
- La placa debe tener exactamente 6 caracteres.

--- Parqueadero Nido de Ruedas ---
1. Ingresar vehículo
2. Retirar vehículo
3. Módulo administrador
4. Salir
        
Seleccione una opción: 3
Ingrese usuario administrador: dmin
Ingrese clave: 1234

Usuario o clave incorrectos.


--- Parqueadero Nido de Ruedas ---
1. Ingresar vehículo
2. Retirar vehículo
3. Módulo administrador
4. Salir
        
Seleccione una opción: 6
Opción inválida.

--- Parqueadero Nido de Ruedas ---
1. Ingresar vehículo
2. Retirar vehículo
3. Módulo administrador
4. Salir
        
Seleccione una opción: 3
Ingrese usuario administrador: admin
Ingrese clave: 1234

--- Reportes Administrador ---
1. T

KeyboardInterrupt: Interrupted by user